In [1]:
from data.data import COVID19India

In [2]:
cin = COVID19India()

In [3]:
df = cin.moh_data(save=False)

In [4]:
df.tail()

,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death
22,Ladakh,13,0,0,0
23,Uttar Pradesh,36,1,11,0
24,Uttarakhand,4,1,0,0
25,West Bengal,9,0,0,1
26,Total number of confirmed cases in India,602,47,43,13


In [7]:
cin.current_update()[0]

{'confirmeddelta': '59',
 'counterforautotimeupdate': '645',
 'deceaseddelta': '2',
 'lastupdatedtime': '26/03/2020 18:16:23',
 'recovereddelta': '2',
 'statesdelta': '0'}

In [27]:
cin.last_update()

'on 26.03.2020 at 10:15 AM'

In [15]:
a, b = cin.StateWise_data()

In [18]:
a.head()

,state_ut,confirmed,active,recovered,deaths,last_updated
0,Total,716,658,45,13,26/03/2020 18:16:23
1,Maharashtra,125,122,1,2,26/03/2020 15:20:23
2,Kerala,137,131,6,0,26/03/2020 18:16:24
3,Karnataka,55,50,3,2,26/03/2020 16:48:24
4,Telangana,44,43,1,0,26/03/2020 14:25:24


In [19]:
b.head()

,state_ut,confirmed,active,recovered,deaths,last_updated
0,Total,59,58,0,1,26/03/2020 18:16:23
1,Maharashtra,3,3,0,0,26/03/2020 15:20:23
2,Kerala,19,19,0,0,26/03/2020 18:16:24
3,Karnataka,4,3,0,1,26/03/2020 16:48:24
4,Telangana,3,3,0,0,26/03/2020 14:25:24


In [20]:
c = cin.state_district_data()

In [21]:
c.head()

,State_UT,District,Confirmed,Active,Deaths,Recovered
0,Kerala,Thrissur,5,3,0,2
1,Kerala,Alappuzha,2,1,0,1
2,Kerala,Kasaragod,46,45,0,1
3,Kerala,Pathanamthitta,12,12,0,0
4,Kerala,Kannur,23,22,0,1


In [22]:
d = cin.timeseries_data()

In [23]:
d.head()

,dailyconfirmed,dailydeceased,dailyrecovered,date,totalconfirmed,totaldeceased,totalrecovered
0,1,0,0,30 January,1,0,0
1,0,0,0,31 January,1,0,0
2,0,0,0,01 February,1,0,0
3,1,0,0,02 February,2,0,0
4,1,0,0,03 February,3,0,0


In [35]:
d['date'] = pd.to_datetime(d['date'] + '2020')

In [37]:
d = d[d['dailyconfirmed'] != ""]

In [38]:
d.tail()

,dailyconfirmed,dailydeceased,dailyrecovered,date,totalconfirmed,totaldeceased,totalrecovered
51,76,0,0,2020-03-21,334,4,23
52,69,3,0,2020-03-22,403,7,23
53,102,2,2,2020-03-23,505,9,25
54,66,1,15,2020-03-24,571,10,40
55,86,1,3,2020-03-25,657,11,43


0    2020-01-30
1    2020-01-31
2    2020-02-01
3    2020-02-02
4    2020-02-03
5    2020-02-04
6    2020-02-05
7    2020-02-06
8    2020-02-07
9    2020-02-08
10   2020-02-09
11   2020-02-10
12   2020-02-11
13   2020-02-12
14   2020-02-13
15   2020-02-14
16   2020-02-15
17   2020-02-16
18   2020-02-17
19   2020-02-18
20   2020-02-19
21   2020-02-20
22   2020-02-21
23   2020-02-22
24   2020-02-23
25   2020-02-24
26   2020-02-25
27   2020-02-26
28   2020-02-27
29   2020-02-28
30   2020-02-29
31   2020-03-01
32   2020-03-02
33   2020-03-03
34   2020-03-04
35   2020-03-05
36   2020-03-06
37   2020-03-07
38   2020-03-08
39   2020-03-09
40   2020-03-10
41   2020-03-11
42   2020-03-12
43   2020-03-13
44   2020-03-14
45   2020-03-15
46   2020-03-16
47   2020-03-17
48   2020-03-18
49   2020-03-19
50   2020-03-20
51   2020-03-21
52   2020-03-22
53   2020-03-23
54   2020-03-24
55   2020-03-25
56   2020-03-26
Name: date, dtype: datetime64[ns]

In [25]:
e = cin.tests()

In [26]:
e

,totalindividualstested,totalpositivecases,totalsamplestested,updatetimestamp
0,5900,78,6500,13/3/2020 00:00
1,12235,150,13125,18/3/2020 18:00
2,12426,168,13316,19/3/2020 10:00
3,13285,182,14175,19/3/2020 18:00
4,13486,206,14376,20/3/2020 10:00
5,14514,236,15404,20/3/2020 18:00
6,14811,271,15701,21/3/2020 10:00
7,16021,315,16911,21/3/2020 18:00
8,16109,341,16999,22/3/2020 10:00
9,17237,396,18127,22/3/2020 18:00


In [30]:
import pandas as pd

In [31]:
from pathlib import Path

In [32]:
import pandas as pd
import numpy as np
import requests

markdown_file = open('help.md')
file_content = markdown_file.read()
md = markdown.Markdown(extensions=['tables', 'codehilite', 'fenced_code'])
pg = md.convert(file_content)